In [ ]:
# Notebook to retrieve and save the observed weather data

from datetime import datetime

import meteostat
import opts
import xarray as xr
from climepi._xcdat import BoundsAccessor, center_times  # noqa

# # Get options
options = opts.get_opts()
data_base_dir = options["data_base_dir"]

In [ ]:
stations_all = meteostat.Stations().fetch()
station_dict = {
    "03772": "London",  # "London Heathrow Airport",
    "07149": "Paris",  # "Paris-Orly Airport",
    "17060": "Istanbul",  # "Istanbul Ataturk Airport",
    "68816": "Cape Town",  # "Cape Town International Airport",
    "72295": "Los Angeles",  # "Los Angeles International Airport",
}

for station, location in station_dict.items():
    print(
        f"For location {location}, using data from weather station: {stations_all.loc[station]['name']}"
    )

In [3]:
df = meteostat.Daily(
    loc=list(station_dict),
    start=datetime(2020, 1, 1),
    end=datetime(2020, 12, 31),
).fetch()[["tavg"]]
ds = (
    xr.Dataset.from_dataframe(df)
    .rename_vars(tavg="temperature")
    .assign_coords(location=("station", list(station_dict.values())))
    .swap_dims(station="location")
    .drop_vars("station")
)

In [ ]:
ds = center_times(ds.bounds.add_time_bounds(method="freq", freq="day"))
ds["time"].encoding["calendar"] = "standard"
ds["time"].encoding["units"] = "hours since 2000-01-01 00:00:00"
ds

In [6]:
ds.to_netcdf(data_base_dir / "weather_2020.nc")